Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import qiskit 
from qiskit_algorithms.optimizers import SPSA
import sys
sys.path.append('../')
import models
from models import CNN_Simple as CNN_Simple
import torchvision
import torchvision.transforms as transforms

Global Variables:

In [ ]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

data = training_set.data.type(torch.double)[:, None, :, :]
target = training_set.targets

Code:

In [ ]:
def load_params(model, param_tensor):
    index = 0
    for param in model.parameters():
        param_length = param.numel()
        param.data = param_tensor[index:index + param_length].view(param.size())
        index += param_length

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def batch_loss(batch_input, batch_output):
  def loss_f(weights):
    model = CNN_Simple()
    load_params(model, torch.tensor(weights))
    prediction = model.forward(batch_input)
    return criterion(prediction, batch_output.type(torch.long)).item()
  
  return loss_f

In [ ]:
class ShowSteps:
  def __init__(self):
    self.losses = []

  def callback(self, n, weights, loss_f, step_size, accepted):
    self.losses.append(loss_f)

In [247]:
steps = ShowSteps()
model = CNN_Simple()
param_list = np.random.randn(sum(p.numel() for p in model.parameters())).astype(np.float32)
param_tensor = torch.tensor(param_list)
losses = []

for batch_input, batch_output in training_loader:
  our_spsa = SPSA(maxiter=1, learning_rate=0.001, perturbation=0.1, callback=steps.callback)
  result = our_spsa.minimize(batch_loss(batch_input.type(torch.double), batch_output.type(torch.double)), param_tensor)
  param_tensor = result.x
  losses.append(result.fun)
  print(losses[-1])
  #print("Minimum =", param_tensor)

34419.225195584506
3815529848.2166543
4.378163724398987e+25
1.19981401926982e+72
2.500645871960841e+72
2.613364353482993e+72
2.4247370422842704e+72
1.334343774768753e+72
6.728293613236308e+71
1.8636870075453633e+72
1.1024687484987241e+72
2.5394735894313286e+72
3.965959678782764e+72


KeyboardInterrupt: 